# 02 – Data Cleaning & Feature Engineering

**Notebook Name:** `02_Data_Cleaning_Feature_Engineering.ipynb`

## Objectives
- Load raw data from `inputs/datasets/raw/HousePricesRecords.csv`.
- Handle nulls and duplicates.
- Parse dates into separate Year and Month features.
- Map and one-hot encode key categorical fields.
- Save the cleaned, feature-engineered dataset for modeling.

## Inputs
- `inputs/datasets/raw/HousePricesRecords.csv` (raw Price-Paid records).

## Outputs
- `outputs/datasets/collection/HousePricesRecords_clean.csv` (cleaned and feature-engineered).

## Additional Comments
- Follows CRISP-DM Phase 3: Data Preparation.
- This cleaned CSV is the source of truth for subsequent analysis notebooks and the Streamlit app.

## Import Required Libraries
Loading packages for data manipulation and file operations.

In [1]:
import pandas as pd
import numpy as np
import os

##  Load Raw Data Sample
Read a subset of rows in chunks to manage memory.

We read in chunks of 200,000 rows and keep the newest 1,000 entries overall for initial inspection.

In [ ]:
input_path = "../inputs/datasets/raw/price_paid_records.csv"
chunksize = 200_000    
topn      = 1_000      
latest = None

for chunk in pd.read_csv(
    input_path,
    usecols=[
        "Price","Date of Transfer","Property Type","Old/New",
        "Duration","Town/City","County","PPDCategory Type"
    ],
    parse_dates=["Date of Transfer"],
    chunksize=chunksize,
    low_memory=False
):
    # for this chunk, keep its top `topn` newest rows
    chunk_top = chunk.nlargest(topn, "Date of Transfer")

    if latest is None:
        latest = chunk_top
    else:
        # merge with previous bests, then re-take the top `topn`
        latest = pd.concat([latest, chunk_top]).nlargest(topn, "Date of Transfer")
    

df_chunk = latest.reset_index(drop=True)

### Preview Sample
Ensure data loaded correctly; limited to 1,000 rows due to memory constraints.

In [ ]:
df_chunk.head()

## Inspect Data Types & Distributions
Verify dtypes and explore categorical value counts.

In [ ]:
print("Column Data Types:")
print(df_chunk.dtypes)

categorical_cols = ["Property Type", "Old/New", "Duration", "County", "Town/City"]
for col in categorical_cols:
    print(f"\nValue counts for {col}:")
    print(df_chunk[col].value_counts())

## Clean Missing and Irrelevant Data
Remove rows with missing critical fields and drop unused columns.

In [ ]:
print("Missing values in each column:")
print(df_chunk.isnull().sum())

df_chunk.dropna(subset=["Price", "Date of Transfer"], inplace=True)
print(f"Shape after dropping missing price/date: {df_chunk.shape}")

## Date Feature Engineering
Convert date to datetime, extract year and month, and filter last 3 years.

In [ ]:
df_chunk.drop(columns=[
    "Transaction unique identifier",
    "District",
    "Record Status - monthly file only"
], inplace=True, errors='ignore')


df_chunk["Date of Transfer"] = pd.to_datetime(df_chunk["Date of Transfer"], errors='coerce')

## Encode Categorical Features
Map binary and one-hot encode needful fields.

In [ ]:

df_chunk["Year"] = df_chunk["Date of Transfer"].dt.year
df_chunk["Month"] = df_chunk["Date of Transfer"].dt.month

max_date    = df_chunk["Date of Transfer"].max()
cutoff_date = max_date - pd.DateOffset(years=3)

# 2) filter to only those transfers within the last 3 years
before_rows = df_chunk.shape[0]
df_chunk    = df_chunk[df_chunk["Date of Transfer"] >= cutoff_date]
after_rows  = df_chunk.shape[0]

df_chunk["Old/New"] = df_chunk["Old/New"].map({'N': 0, 'Y': 1})
df_chunk["Duration"] = df_chunk["Duration"].map({'F': 1, 'L': 0})
df_chunk = pd.get_dummies(df_chunk, columns=["Property Type"], prefix="Property")

print(
    f"Filtered out {before_rows - after_rows} rows; "
    f"remaining {after_rows} rows from {cutoff_date.date()} → {max_date.date()}"
)


## Remove Duplicates & Add Aggregates
Ensure uniqueness and compute region/county-level features.

In [ ]:

df_chunk.drop_duplicates(inplace=True)
df_chunk.reset_index(drop=True, inplace=True)

## Price Transformation & Outlier Removal
Log-transform target and remove extreme outliers calculate the value at the 99.5th percentile of LogPrice.

In [ ]:
import numpy as np


df_chunk['LogPrice'] = np.log1p(df_chunk['Price'])
cutoff = df_chunk['LogPrice'].quantile(0.995)
before = df_chunk.shape[0]

df_chunk = df_chunk[df_chunk['LogPrice'] <= cutoff]
after = df_chunk.shape[0]
print(f"Dropped {before-after} outliers; new shape: {df_chunk.shape}")

## Save Cleaned Data
Write cleaned DataFrame to CSV for downstream notebooks.

In [ ]:

output_path = 'outputs/datasets/collection/HousePricesRecords_clean.csv'
import os
os.makedirs(os.path.dirname(output_path), exist_ok=True)


df_chunk.to_csv(output_path, index=False)
print(f"Saved cleaned data to {output_path}")